<hr/>
指定要處理的Excel文件和sheet<br>
刪除有在不同行出現的重複值，只保留組內元素最少的那個<br>
以Excel文件名稱+"_modified"回存處理好重覆值的檔案<br>
<hr/>

In [ ]:
import pandas as pd
from collections import Counter

# 假设每个Excel文件中只有一个工作表，并且工作表名称为Sheet1
sheet_name_A = 'Euclidean_5_Sil_Cos_-0.3784_Sil'

# 读取Excel文件
file_path = 'clustering_results_hdbscan_0628_Processed.xlsx'  # 将此路径替换为你的Excel文件路径
df = pd.read_excel(file_path, sheet_name=sheet_name_A, header=None)  # 读取Excel文件，无标题行

# 展平DataFrame并统计每个值出现的次数
values = df.values.flatten()
counter = Counter(values)

# 查找重复值并打印
duplicates = {item: count for item, count in counter.items() if count > 1}

# 打印重复值及其出现次数
# for value, count in duplicates.items():
#     print(f"Value: {value}, Count: {count}")

# 删除重复值，只保留所在列中元素最少的那列中的一个
count = 0
for value in duplicates:
    if value == '-------' or pd.isna(value):
        continue
    # 获取所有重复值的位置
    positions = [(row_idx, col_idx) for row_idx, row in df.iterrows() for col_idx, cell in enumerate(row) if cell == value]
    
    # 获取每个重复值所在列的元素数量
    col_element_counts = {col_idx: df.iloc[:, col_idx].count() for _, col_idx in positions}
    
    # 按元素数量从少到多排序位置
    positions_sorted = sorted(positions, key=lambda pos: col_element_counts[pos[1]])
    
    # 保留元素数量最少的列中的一个，删除其他的
    positions_to_delete = positions_sorted[1:]
    for row_idx, col_idx in positions_to_delete:
        df.iat[row_idx, col_idx] = None  # 将多余的重复值设为None（即删除）
    count += 1
    print(count)

# 保存修改后的Excel文件
output_file_path = sheet_name_A + "_modified.xlsx"
df.to_excel(output_file_path, header=False, index=False)


<hr/>
指定要處理的Excel文件<br>
刪除有在不同行出現的重複值，隨機選擇要保留的那個<br>
<hr/>

In [ ]:
import pandas as pd
from collections import Counter
import random

# 读取Excel文件
file_path = 'clustering_results_C_K_390.xlsx'
df = pd.read_excel(file_path, header=None)

# 展平DataFrame并统计每个值出现的次数
values = df.values.flatten()
counter = Counter(values)

# 查找重复值
duplicates = {item: count for item, count in counter.items() if count > 1}

# 打印重复值及其出现次数
for value, count in duplicates.items():
    print(f"Value: {value}, Count: {count}")

# 删除重复值，只保留一个
for value in duplicates:
    positions = [(row_idx, col_idx) for row_idx, row in df.iterrows() for col_idx, cell in enumerate(row) if cell == value]
    positions_to_delete = random.sample(positions, len(positions) - 1)
    for row_idx, col_idx in positions_to_delete:
        df.iat[row_idx, col_idx] = None  # 将多余的重复值设为None（即删除）

# 保存修改后的Excel文件
df.to_excel('clustering_results_C_K_390_modified.xlsx', header=False, index=False)


<hr/>
小功能<br>
<hr/>

In [ ]:
import pandas as pd

# 读取Excel文件A和B
file_a = 'test_5.xlsx'
file_b = 'clustering_results_hdbscan_0624_.xlsx'

# 假设每个Excel文件中只有一个工作表，并且工作表名称为Sheet1
sheet_name_A = '工作表1'
sheet_name_B = 'Euclidean_MinCluster_5_Sil_Cos_'

# 读取Excel文件A和B的内容
df_a = pd.read_excel(file_a, sheet_name=sheet_name_A)
df_b = pd.read_excel(file_b, sheet_name=sheet_name_B)

# 用于存储文件A中有但文件B中没有的单元格
diff_cells = []

# 比较两个DataFrame
for i in range(df_a.shape[0]):  # 遍历行
    for j in range(df_a.shape[1]):  # 遍历列
        if i < df_b.shape[0] and j < df_b.shape[1]:  # 在文件B的范围内
            value_a = df_a.iat[i, j]
            value_b = df_b.iat[i, j]
            if pd.isnull(value_a) and pd.isnull(value_b):
                continue
            if not pd.isnull(value_a) and not pd.isnull(value_b):
                if value_a != value_b:
                    diff_cells.append((i, j, value_a, value_b))
            elif pd.isnull(value_a) or pd.isnull(value_b):
                diff_cells.append((i, j, value_a, value_b))
        else:  # 超出文件B的范围
            value_a = df_a.iat[i, j]
            diff_cells.append((i, j, value_a, None))

# 打印文件A中有但文件B中没有的单元格
print("Cells in A but not in B:")
for diff in diff_cells:
    print(f"Row: {diff[0]}, Column: {diff[1]}, A: {diff[2]}, B: {diff[3]}")

# 如果需要将结果保存到一个新的Excel文件
diff_df = pd.DataFrame(diff_cells, columns=['Row', 'Column', 'Value in A', 'Value in B'])
diff_df.to_excel('diff_cells.xlsx', index=False)


<hr/>
小功能<br>
<hr/>

In [ ]:
import pandas as pd

# 读取Excel文件A和B
file_a = 'test_5.xlsx'
file_b = 'clustering_results_hdbscan_4_1.xlsx'

# 假设每个Excel文件中只有一个工作表，并且工作表名称为Sheet1
sheet_name_A = '工作表1'
sheet_name_B = '工作表1'

# 读取Excel文件A和B的内容
df_a = pd.read_excel(file_a, sheet_name=sheet_name_A)
df_b = pd.read_excel(file_b, sheet_name=sheet_name_B)

# 将所有单元格的值转换为列表
values_a = df_a.values.flatten().tolist()
values_b = df_b.values.flatten().tolist()

# 找出文件A中有但文件B中没有的值
diff_values = set(values_a) - set(values_b)

# 打印文件A中有但文件B中没有的值
print("Values in A but not in B:")
for value in diff_values:
    print(value)

# 如果需要将结果保存到一个新的Excel文件
diff_df = pd.DataFrame(list(diff_values), columns=['Values in A but not in B'])
diff_df.to_excel('diff_values.xlsx', index=False)
